In [ ]:
%pip install wildlife-datasets

In [ ]:
%pip install wildlife-tools

In [ ]:
%pip install timm

In [ ]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
import timm
import numpy as np
from PIL import Image
import torchvision.transforms as T
import cv2  # Import OpenCV for SIFT

from sklearn.metrics import precision_score, recall_score, f1_score

from wildlife_tools.data import WildlifeDataset
from wildlife_datasets.datasets import MacaqueFaces

from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from wildlife_tools.features import SIFTFeatures


from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier



**CowDataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')

# Load dataset metadata
metadata_CZoo = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)

In [ ]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')

# Load dataset metadata
metadata_CZoo = datasets.CowDataset('../data/CowDataset')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo = SIFTFeatures()

In [ ]:
query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

In [ ]:
print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)

In [ ]:
classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_CZoo * 100))



In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")

**CZoo Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_CZoo = datasets.CZoo('../data/CZoo')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo = SIFTFeatures()

query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')


In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_CZoo * 100))


In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")


**IPanda50 Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/Ipanda50')

# Load dataset metadata
metadata_Ipanda50 = datasets.IPanda50('../data/Ipanda50')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_Ipanda50.df, metadata_Ipanda50.root, transform=transform)
dataset_database_Ipanda50 = WildlifeDataset(metadata_Ipanda50.df.iloc[100:], metadata_Ipanda50.root, transform=transform)
dataset_query_Ipanda50 = WildlifeDataset(metadata_Ipanda50.df.iloc[:100], metadata_Ipanda50.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_Ipanda50 = SIFTFeatures()

query_Ipanda50, database_Ipanda50 = extractor_Ipanda50(dataset_query_Ipanda50), extractor_Ipanda50(dataset_database_Ipanda50)

print(f'First 5 query features shape: {[i.shape for i in query_Ipanda50[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_Ipanda50[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_Ipanda50, database_Ipanda50)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_Ipanda50 = KnnClassifier(k=1, database_labels=dataset_database_Ipanda50.labels_string)
predictions_Ipanda50 = classifier_Ipanda50(sim)

print("Predictions for 100 test Images:-\n",predictions_Ipanda50)
accuracy_Ipanda50 = np.mean(dataset_query_Ipanda50.labels_string == predictions_Ipanda50)
print("Accuracy on Ipanda50 data: {:.2f}%".format(accuracy_Ipanda50 * 100))

In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_Ipanda50.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_Ipanda50.labels_string)

acc = sum(preds == dataset_query_Ipanda50.labels_string) / len(dataset_query_Ipanda50.labels_string)
print('\n Accuracy: ', acc*100,"%")


**LionData Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')

# Load dataset metadata
metadata_LionData = datasets.LionData('../data/LionData')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_LionData.df, metadata_LionData.root, transform=transform)
dataset_database_LionData = WildlifeDataset(metadata_LionData.df.iloc[100:], metadata_LionData.root, transform=transform)
dataset_query_LionData = WildlifeDataset(metadata_LionData.df.iloc[:100], metadata_LionData.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_LionData = SIFTFeatures()

query_LionData, database_LionData = extractor_LionData(dataset_query_LionData), extractor_LionData(dataset_database_LionData)

print(f'First 5 query features shape: {[i.shape for i in query_LionData[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_LionData[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.6])
sim = similarity(query_LionData, database_LionData)[0.6]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_LionData = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
predictions_LionData = classifier_LionData(sim)

print("Predictions for 100 test Images:-\n",predictions_LionData)
accuracy_LionData = np.mean(dataset_query_LionData.labels_string == predictions_LionData)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_LionData * 100))


In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_LionData.labels_string)

acc = sum(preds == dataset_query_LionData.labels_string) / len(dataset_query_LionData.labels_string)
print('\n Accuracy: ', acc*100,"%")


**MacaqueFaces Dataset**


In [ ]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')

# Load dataset metadata
metadata_MacaqueFaces = datasets.MacaqueFaces('../data/MacaqueFaces')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_MacaqueFaces.df, metadata_MacaqueFaces.root, transform=transform)

dataset_database_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[100:], metadata_MacaqueFaces.root, transform=transform)
dataset_query_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[:100], metadata_MacaqueFaces.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_MacaqueFaces = SIFTFeatures()

query_MacaqueFaces, database_MacaqueFaces = extractor_MacaqueFaces(dataset_query_MacaqueFaces), extractor_MacaqueFaces(dataset_database_MacaqueFaces)

print(f'First 5 query features shape: {[i.shape for i in query_MacaqueFaces[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_MacaqueFaces[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_MacaqueFaces, database_MacaqueFaces)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_MacaqueFaces = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
predictions_MacaqueFaces = classifier_MacaqueFaces(sim)

print("Predictions for 100 test Images:-\n",predictions_MacaqueFaces)
accuracy_MacaqueFaces = np.mean(dataset_query_MacaqueFaces.labels_string == predictions_MacaqueFaces)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_MacaqueFaces * 100))

In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_MacaqueFaces.labels_string)

acc = sum(preds == dataset_query_MacaqueFaces.labels_string) / len(dataset_query_MacaqueFaces.labels_string)
print('\n Accuracy: ', acc*100,"%")

**NyalaDataSet Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')

# Load dataset metadata
metadata_NyalaData = datasets.NyalaData('../data/NyalaData')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_NyalaData.df, metadata_NyalaData.root, transform=transform)
dataset_database_NyalaData = WildlifeDataset(metadata_NyalaData.df.iloc[100:], metadata_NyalaData.root, transform=transform)
dataset_query_NyalaData = WildlifeDataset(metadata_NyalaData.df.iloc[:100], metadata_NyalaData.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_NyalaData = SIFTFeatures()

query_NyalaData, database_NyalaData = extractor_NyalaData(dataset_query_NyalaData), extractor_NyalaData(dataset_database_NyalaData)

print(f'First 5 query features shape: {[i.shape for i in query_NyalaData[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_NyalaData[:5]]}')


In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_NyalaData, database_NyalaData)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_NyalaData = KnnClassifier(k=1, database_labels=dataset_database_NyalaData.labels_string)
predictions_NyalaData = classifier_NyalaData(sim)

print("Predictions for 100 test Images:-\n",predictions_NyalaData)
accuracy_NyalaData = np.mean(dataset_query_NyalaData.labels_string == predictions_NyalaData)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_NyalaData * 100))

In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_NyalaData.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_NyalaData.labels_string)

acc = sum(preds == dataset_query_NyalaData.labels_string) / len(dataset_query_NyalaData.labels_string)
print('\n Accuracy: ', acc*100,"%")

**StripeSpotter Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')

# Load dataset metadata
metadata_StripeSpotter = datasets.StripeSpotter('../data/StripeSpotter')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_StripeSpotter.df, metadata_StripeSpotter.root, transform=transform)
dataset_database_StripeSpotter = WildlifeDataset(metadata_StripeSpotter.df.iloc[100:], metadata_StripeSpotter.root, transform=transform)
dataset_query_StripeSpotter = WildlifeDataset(metadata_StripeSpotter.df.iloc[:100], metadata_StripeSpotter.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_StripeSpotter = SIFTFeatures()

query_StripeSpotter, database_StripeSpotter = extractor_StripeSpotter(dataset_query_StripeSpotter), extractor_StripeSpotter(dataset_database_StripeSpotter)

print(f'First 5 query features shape: {[i.shape for i in query_StripeSpotter[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_StripeSpotter[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_StripeSpotter, database_StripeSpotter)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_StripeSpotter = KnnClassifier(k=1, database_labels=dataset_database_StripeSpotter.labels_string)
predictions_StripeSpotter = classifier_StripeSpotter(sim)

print("Predictions for 100 test Images:-\n",predictions_StripeSpotter)
accuracy_StripeSpotter = np.mean(dataset_query_StripeSpotter.labels_string == predictions_StripeSpotter)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_StripeSpotter * 100))

In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_StripeSpotter.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_StripeSpotter.labels_string)

acc = sum(preds == dataset_query_StripeSpotter.labels_string) / len(dataset_query_StripeSpotter.labels_string)
print('\n Accuracy: ', acc*100,"%")

# **ON NEW UNSEEN DATASETS**

**DogFaceNet**

In [ ]:
# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata_DogFaceNet = datasets.DogFaceNet('../data/DogFaceNet')


# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_DogFaceNet.df, metadata_DogFaceNet.root, transform=transform)
dataset_database_DogFaceNet = WildlifeDataset(metadata_DogFaceNet.df.iloc[100:], metadata_DogFaceNet.root, transform=transform)
dataset_query_DogFaceNet = WildlifeDataset(metadata_DogFaceNet.df.iloc[:100], metadata_DogFaceNet.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_DogFaceNet = SIFTFeatures()

query_DogFaceNet, database_DogFaceNet = extractor_DogFaceNet(dataset_query_DogFaceNet), extractor_DogFaceNet(dataset_database_DogFaceNet)

print(f'First 5 query features shape: {[i.shape for i in query_DogFaceNet[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_DogFaceNet[:5]]}')


In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_DogFaceNet, database_DogFaceNet)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_DogFaceNet = KnnClassifier(k=1, database_labels=dataset_database_DogFaceNet.labels_string)
predictions_DogFaceNet = classifier_DogFaceNet(sim)

print("Predictions for 100 test Images:-\n",predictions_DogFaceNet)
accuracy_DogFaceNet = np.mean(dataset_query_DogFaceNet.labels_string == predictions_DogFaceNet)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_DogFaceNet * 100))


In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_DogFaceNet.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_DogFaceNet.labels_string)

acc = sum(preds == dataset_query_DogFaceNet.labels_string) / len(dataset_query_DogFaceNet.labels_string)
print('\n Accuracy: ', acc*100,"%")